In [ ]:
```json
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import cv2\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import os\n",
    "import glob\n",
    "import pandas as pd\n",
    "import time\n",
    "from collections import defaultdict\n",
    "\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define paths for Haar Cascades (download from OpenCV's GitHub repository if not available)\n",
    "# Ensure these files exist in the specified paths or update the paths accordingly.\n",
    "face_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'\n",
    "eye_cascade_path = cv2.data.haarcascades + 'haarcascade_eye.xml'\n",
    "\n",
    "# Define base directory for sample data\n",
    "# PLEASE CREATE THIS DIRECTORY STRUCTURE AND ADD SAMPLE IMAGES:\n",
    "# ./gaze_data/\n",
    "#  |- straight/\n",
    "#  |   |- img1.jpg\n",
    "#  |   |- img2.png\n",
    "#  |   ...\n",
    "#  |- left/\n",
    "#  |   |- img3.jpg\n",
    "#  |   ...\n",
    "#  |- right/\n",
    "#  |   |- img4.jpg\n",
    "#  |   ...\n",
    "#  |- away/  (e.g., eyes closed, looking far up/down)\n",
    "#  |   |- img5.jpg\n",
    "#  |   ...\n",
    "#  |- no_face/\n",
    "#  |   |- img6.jpg\n",
    "#  |   ...\n",
    "data_base_dir = './gaze_data/'\n",
    "categories = ['straight', 'left', 'right', 'away', 'no_face']\n",
    "\n",
    "# Create directories if they don't exist (user still needs to add images)\n",
    "for category in categories:\n",
    "    os.makedirs(os.path.join(data_base_dir, category), exist_ok=True)\n",
    "\n",
    "print(f\"Haar Cascade paths:\")\n",
    "print(f\"Face: {face_cascade_path}\")\n",
    "print(f\"Eye: {eye_cascade_path}\")\n",
    "print(f\"\\nData directory structure created/verified in: {os.path.abspath(data_base_dir)}\")\n",
    "print(\"Please populate the subdirectories with sample images.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 1. Data Loading and Inspection\n",
    "def load_image_paths(base_dir, categories):\n",
    "    image_paths = defaultdict(list)\n",
    "    for category in categories:\n",
    "        search_path = os.path.join(base_dir, category, '*')\n",
    "        paths = glob.glob(search_path)\n",
    "        # Filter for common image extensions\n",
    "        image_paths[category] = [p for p in paths if p.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]\n",
    "    return image_paths\n",
    "\n",
    "all_image_paths = load_image_paths(data_base_dir, categories)\n",
    "\n",
    "total_images = sum(len(paths) for paths in all_image_paths.values())\n",
    "print(f\"Found {total_images} images across {len(categories)} categories.\")\n",
    "\n",
    "if total_images == 0:\n",
    "    print(\"\\nWARNING: No images found. Please add images to the directories under\", data_base_dir)\n",
    "else:\n",
    "    print(\"\\nImage counts per category:\")\n",
    "    for category, paths in all_image_paths.items():\n",
    "        print(f\"- {category}: {len(paths)}\")\n",
    "\n",
    "    # Inspect a sample image\n",
    "    first_category = next(iter(all_image_paths)) if total_images > 0 else None\n",
    "    if first_category and all_image_paths[first_category]:\n",
    "        sample_image_path = all_image_paths[first_category][0]\n",
    "        print(f\"\\nLoading sample image: {sample_image_path}\")\n",
    "        try:\n",
    "            sample_img = cv2.imread(sample_image_path)\n",
    "            if sample_img is not None:\n",
    "                print(f\"Sample image shape: {sample_img.shape}\") \n",
    "                print(f\"Sample image dtype: {sample_img.dtype}\")\n",
    "                \n",
    "                # Display sample image\n",
    "                plt.figure(figsize=(5, 5))\n",
    "                plt.imshow(cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB))\n",
    "                plt.title(f\"Sample Image ({first_category})\")\n",
    "                plt.axis('off')\n",
    "                plt.show()\n",
    "            else:\n",
    "                print(f\"Error: Could not load sample image {sample_image_path}\")\n",
    "        except Exception as e:\n",
    "            print(f\"Error loading or displaying sample image {sample_image_path}: {e}\")\n",
    "    else:\n",
    "        print(\"\\nCannot display sample image as no images were found.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load Haar Cascades\n",
    "face_cascade = cv2.CascadeClassifier(face_cascade_path)\n",
    "eye_cascade = cv2.CascadeClassifier(eye_cascade_path)\n",
    "\n",
    "if face_cascade.empty():\n",
    "    print(f\"Error loading face cascade from {face_cascade_path}\")\n",
    "if eye_cascade.empty():\n",
    "    print(f\"Error loading eye cascade from {eye_cascade_path}\")\n",
    "\n",
    "print(\"Haar cascades loaded.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 2. Exploratory Data Analysis (EDA) with Plots\n",
    "\n",
    "def detect_faces_eyes(image_path, face_cascade, eye_cascade):\n",
    "    img = cv2.imread(image_path)\n",
    "    if img is None:\n",
    "        print(f\"Warning: Could not read image {image_path}\")\n",
    "        return None, 0, 0, [], []\n",
    "        \n",
    "    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)\n",
    "    \n",
    "    # Detect faces\n",
    "    # Adjust scaleFactor and minNeighbors for different results\n",
    "    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))\n",
    "    \n",
    "    detected_eyes_list = []\n",
    "    face_rois = []\n",
    "\n",
    "    img_with_detections = img.copy()\n",
    "    \n",
    "    for (x, y, w, h) in faces:\n",
    "        cv2.rectangle(img_with_detections, (x, y), (x+w, y+h), (255, 0, 0), 2) # Blue for faces\n",
    "        roi_gray = gray[y:y+h, x:x+w]\n",
    "        roi_color = img_with_detections[y:y+h, x:x+w]\n",
    "        face_rois.append((x, y, w, h))\n",
    "        \n",
    "        # Detect eyes within each face ROI\n",
    "        # Adjust scaleFactor and minNeighbors for eyes\n",
    "        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=4, minSize=(15, 15))\n",
    "        \n",
    "        face_eyes = []\n",
    "        for (ex, ey, ew, eh) in eyes:\n",
    "            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2) # Green for eyes\n",
    "            # Store eye coordinates relative to the full image\n",
    "            face_eyes.append((x + ex, y + ey, ew, eh))\n",
    "            \n",
    "        detected_eyes_list.append(face_eyes)\n",
    "            \n",
    "    num_faces = len(faces)\n",
    "    num_eyes = sum(len(eye_list) for eye_list in detected_eyes_list)\n",
    "    \n",
    "    return img_with_detections, num_faces, num_eyes, face_rois, detected_eyes_list\n",
    "\n",
    "# Process all images and store detection results\n",
    "detection_results = []\n",
    "processed_images = {} # Store images with detections for later display\n",
    "\n",
    "start_time = time.time()\n",
    "img_count = 0\n",
    "for category, paths in all_image_paths.items():\n",
    "    if not paths:\n",
    "        continue\n",
    "    print(f\"Processing category: {category}...\")\n",
    "    for img_path in paths:\n",
    "        img_count += 1\n",
    "        img_with_detections, n_faces, n_eyes, faces_coords, eyes_coords_list = detect_faces_eyes(img_path, face_cascade, eye_cascade)\n",
    "        if img_with_detections is not None:\n",
    "            detection_results.append({\n",
    "                'path': img_path,\n",
    "                'category': category,\n",
    "                'num_faces': n_faces,\n",
    "                'num_eyes': n_eyes,\n",
    "                'faces_coords': faces_coords, # List of (x,y,w,h) for faces\n",
    "                'eyes_coords_list': eyes_coords_list # List of lists of (x,y,w,h) for eyes per face\n",
    "            })\n",
    "            processed_images[img_path] = img_with_detections\n",
    "        if img_count % 20 == 0:\n",
    "             print(f\"  Processed {img_count}/{total_images} images...\")\n",
    "\n",
    "end_time = time.time()\n",
    "print(f\"\\nDetection finished in {end_time - start_time:.2f} seconds.\")\n",
    "\n",
    "if not detection_results:\n",
    "     print(\"\\nNo images were processed. Cannot perform EDA.\")\n",
    "else:\n",
    "    # Create a DataFrame for easier analysis\n",
    "    df_results = pd.DataFrame(detection_results)\n",
    "    print(\"\\nDetection Results Summary:\")\n",
    "    print(df_results.head())\n",
    "    print(f\"\\nTotal processed results: {len(df_results)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# EDA Plots\n",
    "if not df_results.empty:\n",
    "    # Plot distribution of detected faces per image\n",
    "    plt.figure(figsize=(12, 5))\n",
    "    plt.subplot(1, 2, 1)\n",
    "    df_results['num_faces'].value_counts().sort_index().plot(kind='bar')\n",
    "    plt.title('Distribution of Detected Faces per Image')\n",
    "    plt.xlabel('Number of Faces Detected')\n",
    "    plt.ylabel('Frequency')\n",
    "    plt.grid(axis='y', linestyle='--')\n",
    "\n",
    "    # Plot distribution of detected eyes per image\n",
    "    plt.subplot(1, 2, 2)\n",
    "    df_results['num_eyes'].value_counts().sort_index().plot(kind='bar')\n",
    "    plt.title('Distribution of Detected Eyes per Image')\n",
    "    plt.xlabel('Number of Eyes Detected')\n",
    "    plt.ylabel('Frequency')\n",
    "    plt.grid(axis='y', linestyle='--')\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "    # Plot detection counts by category\n",
    "    plt.figure(figsize=(10, 6))\n",
    "    df_results.groupby('category')[['num_faces', 'num_eyes']].mean().plot(kind='bar', ax=plt.gca())\n",
    "    plt.title('Average Detections per Category')\n",
    "    plt.ylabel('Average Count')\n",
    "    plt.xticks(rotation=45)\n",
    "    plt.grid(axis='y', linestyle='--')\n",
    "    plt.show()\n",
    "    \n",
    "    # Display some images with detections\n",
    "    num_samples_to_show = min(5, len(df_results))\n",
    "    if num_samples_to_show > 0:\n",
    "        print(f\"\\nDisplaying {num_samples_to_show} sample images with detections:\")\n",
    "        sample_indices = df_results.sample(num_samples_to_show).index\n",
    "        plt.figure(figsize=(15, 5 * num_samples_to_show // 2))\n",
    "        for i, idx in enumerate(sample_indices):\n",
    "            row = df_results.loc[idx]\n",
    "            img_path = row['path']\n",
    "            img_display = processed_images.get(img_path)\n",
    "            if img_display is not None:\n",
    "                plt.subplot( (num_samples_to_show + 1) // 2, 2, i + 1)\n",
    "                plt.imshow(cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB))\n",
    "                plt.title(f\"Category: {row['category']}, Faces: {row['num_faces']}, Eyes: {row['num_eyes']}\")\n",
    "                plt.axis('off')\n",
    "        plt.tight_layout()\n",
    "        plt.show()\n",
    "else:\n",
    "    print(\"Skipping EDA plots as no detection results are available.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 3. Statistical Analysis\n",
    "\n",
    "if not df_results.empty:\n",
    "    print(\"\\nBasic Statistical Analysis:\")\n",
    "    \n",
    "    # Overall detection rates\n",
    "    total_imgs = len(df_results)\n",
    "    imgs_with_face = len(df_results[df_results['num_faces'] > 0])\n",
    "    imgs_with_one_face = len(df_results[df_results['num_faces'] == 1])\n",
    "    imgs_with_eyes = len(df_results[df_results['num_eyes'] > 0])\n",
    "    imgs_with_two_eyes = len(df_results[df_results['num_eyes'] == 2])\n",
    "    imgs_one_face_two_eyes = len(df_results[(df_results['num_faces'] == 1) & (df_results['num_eyes'] == 2)])\n",
    "\n",
    "    print(f\"- Total images analyzed: {total_imgs}\")\n",
    "    if total_imgs > 0:\n",
    "        print(f\"- Images with at least one face detected: {imgs_with_face} ({imgs_with_face/total_imgs:.2%})\")\n",
    "        print(f\"- Images with exactly one face detected: {imgs_with_one_face} ({imgs_with_one_face/total_imgs:.2%})\")\n",
    "        print(f\"- Images with at least one eye detected: {imgs_with_eyes} ({imgs_with_eyes/total_imgs:.2%})\")\n",
    "        print(f\"- Images with exactly two eyes detected: {imgs_with_two_eyes} ({imgs_with_two_eyes/total_imgs:.2%})\")\n",
    "        print(f\"- Images with exactly one face AND two eyes detected: {imgs_one_face_two_eyes} ({imgs_one_face_two_eyes/total_imgs:.2%})\")\n",
    "\n",
    "    # Analysis per category\n",
    "    print(\"\\nDetection Statistics per Category:\")\n",
    "    stats_per_category = df_results.groupby('category').agg(\n",
    "        total_images=('path', 'count'),\n",
    "        avg_faces=('num_faces', 'mean'),\n",
    "        std_faces=('num_faces', 'std'),\n",
    "        avg_eyes=('num_eyes', 'mean'),\n",
    "        std_eyes=('num_eyes', 'std'),\n",
    "        face_detected_rate=('num_faces', lambda x: (x > 0).mean()),\n",
    "        two_eyes_detected_rate=('num_eyes', lambda x: (x == 2).mean())\n",
    "    ).reset_index()\n",
    "    \n",
    "    # Format percentages\n",
    "    stats_per_category['face_detected_rate'] = stats_per_category['face_detected_rate'].map('{:.2%}'.format)\n",
    "    stats_per_category['two_eyes_detected_rate'] = stats_per_category['two_eyes_detected_rate'].map('{:.2%}'.format)\n",
    "    \n",
    "    print(stats_per_category.to_string(index=False))\n",
    "    \n",
    "else:\n",
    "    print(\"Skipping statistical analysis as no detection results are available.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 4. Feature Engineering Experiments\n",
    "\n",
    "# Feature 1: Relative Eye Position (Simplistic Gaze Heuristic)\n",
    "# We'll estimate gaze based on the number of eyes detected and their horizontal position within the face ROI.\n",
    "\n",
    "def estimate_gaze_direction(face_coords, eyes_coords):\n",
    "    \"\"\"Estimates gaze based on eye detection within a single face ROI.\n",
    "    Args:\n",
    "        face_coords (tuple): (x, y, w, h) of the detected face.\n",
    "        eyes_coords (list): List of (ex, ey, ew, eh) for eyes detected within this face (relative to full image).\n",
    "    Returns:\n",
    "        str: 'center', 'left', 'right', 'away', 'unknown'\n",
    "    \"\"\"\n",
    "    if not face_coords or not eyes_coords:\n",
    "        return 'unknown' # Or 'away' if no eyes implies looking away\n",
    "        \n",
    "    fx, fy, fw, fh = face_coords\n",
    "    num_eyes = len(eyes_coords)\n",
    "    \n",
    "    if num_eyes == 0:\n",
    "        return 'away' # No eyes detected within face might mean looking away/closed\n",
    "        \n",
    "    if num_eyes == 2:\n",
    "        # Sort eyes by x-coordinate to identify left and right eye\n",
    "        eyes_coords.sort(key=lambda eye: eye[0])\n",
    "        left_eye = eyes_coords[0]\n",
    "        right_eye = eyes_coords[1]\n",
    "        \n",
    "        # Calculate center of each eye relative to face center\n",
    "        face_center_x = fx + fw / 2\n",
    "        left_eye_center_x = left_eye[0] + left_eye[2] / 2\n",
    "        right_eye_center_x = right_eye[0] + right_eye[2] / 2\n",
    "        \n",
    "        # Heuristic: Check if eyes are roughly symmetrical around face center\n",
    "        # Normalize positions by face width\n",
    "        left_eye_rel_pos = (left_eye_center_x - fx) / fw\n",
    "        right_eye_rel_pos = (right_eye_center_x - fx) / fw\n",
    "        \n",
    "        # Define thresholds (these need tuning!)\n",
    "        center_threshold_low = 0.15\n",
    "        center_threshold_high = 0.85\n",
    "        symmetry_threshold = 0.2 # Max difference between distance from center\n",
    "\n",
    "        left_dist_from_center = abs((left_eye_center_x - face_center_x) / fw)\n",
    "        right_dist_from_center = abs((right_eye_center_x - face_center_x) / fw)\n",
    "\n",
    "        # Check if both eyes are reasonably positioned within the face width\n",
    "        eyes_centered = (center_threshold_low < left_eye_rel_pos < center_threshold_high) and \\\n",
    "                        (center_threshold_low < right_eye_rel_pos < center_threshold_high)\n",
    "        \n",
    "        # Check if eyes are roughly symmetric\n",
    "        eyes_symmetric = abs(left_dist_from_center - right_dist_from_center) < symmetry_threshold\n",
    "\n",
    "        if eyes_centered and eyes_symmetric:\n",
    "             # Further check if eyes are roughly horizontally aligned (simplistic)\n",
    "             y_diff = abs((left_eye[1] + left_eye[3]/2) - (right_eye[1] + right_eye[3]/2))\n",
    "             if y_diff / fh < 0.15: # Allow some vertical difference\n",
    "                 return 'center'\n",
    "             else:\n",
    "                 return 'away' # Significant vertical difference might indicate tilt or odd detection\n",
    "        elif left_eye_rel_pos < center_threshold_low and right_eye_rel_pos < (1-center_threshold_low):\n",
    "            return 'right' # Both eyes shifted to the left side of the face -> looking right\n",
    "        elif left_eye_rel_pos > (1-center_threshold_high) and right_eye_rel_pos > center_threshold_high:\n",
    "            return 'left' # Both eyes shifted to the right side of the face -> looking left\n",
    "        else:\n",
    "            return 'away' # Asymmetric or oddly positioned eyes\n",
    "\n",
    "    elif num_eyes == 1:\n",
    "        # If only one eye is detected, assume looking away/sideways\n",
    "        eye_x = eyes_coords[0][0] + eyes_coords[0][2] / 2\n",
    "        eye_rel_pos = (eye_x - fx) / fw\n",
    "        if eye_rel_pos < 0.4: # Eye detected is on the left side of face\n",
    "             return 'right'\n",
    "        elif eye_rel_pos > 0.6: # Eye detected is on the right side of face\n",
    "             return 'left'\n",
    "        else:\n